In [1]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle
from recbole.data import create_dataset, data_preparation, Interaction
from recbole.utils import init_logger, get_trainer, get_model, init_seed, set_color
from recbole.utils.case_study import full_sort_topk
import os
import glob
from environ import Env
from pathlib import Path
from sqlalchemy import create_engine

print("create engine!")
# Build paths inside the project like this: BASE_DIR / 'subdir'.
BASE_DIR = Path(os.curdir).resolve().parent
env = Env()
env_path = BASE_DIR / "django/.env"
if env_path.exists():
    with env_path.open("rt", encoding="utf8") as f:
        env.read_env(f, overwrite=True)

dbname = env.get_value('GCPDB_NAME')
user = env.get_value('GCPDB_USER')
pw = env.get_value('GCPDB_PASSWORD')
host = env.get_value('GCPDB_HOST')

# engine 생성
engine = create_engine(f'mysql+mysqldb://{user}:{pw}@{host}:3306/{dbname}?charset=utf8')

topk=500
model_name="EASE"
model_path=None


print('inference start!')
if model_path is None:
    # model_name이 들어가는 pth 파일 중 최근에 생성된 걸로 불러옴
    os.makedirs('saved',exist_ok=True)
    save_path = glob.glob('./saved/*')
    latest_model_path = max(save_path, key=os.path.getctime)
model_path_absolute = Path(latest_model_path).absolute()

create engine!
inference start!


In [3]:
mv_df = pd.read_pickle('../Utils/Pickle/230130_Popular_movie_1192_cwj.pickle')
ml_df = pd.read_csv('../Data/EDA/ml-25m/ratings.csv')

merged_df = pd.merge(ml_df,mv_df,how='inner',on='movieId')

merged_df = merged_df.drop(['rating_x', 'timestamp', 'Contents', 'ko_title',
       'contents_year', 'country', 'ko_genre', 'ko_plot', 'rating_y', 'npop',
       'genres', 'plot', 'directors', 'actors'],axis=1)

mask = merged_df.userId.isin(merged_df.userId.value_counts()[merged_df.userId.value_counts() > 350].index)
interactions_df = merged_df[mask]

interactions_df = interactions_df.sort_values('userId')

interactions_df

,userId,movieId
261,548,296
3955949,548,79702
5902187,548,64969
5898945,548,52245
2193245,548,4447
...,...,...
3047145,162516,60072
2454996,162516,7147
3294910,162516,76251
6121435,162516,82667


In [58]:
interactions_df.columns = ['user_id:token', 'item_id:token']

In [61]:
interactions_df.to_csv('../BaseLine/dataset/train_data/train_data.inter',sep='\t',index=False)